In [1]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain_cohere import CohereEmbeddings
text_loader_kwargs = {"autodetect_encoding": True}

loader = DirectoryLoader(
    "./articles",
    glob="*.txt",
    show_progress=True,
    use_multithreading=True,
    loader_cls=TextLoader,
    loader_kwargs=text_loader_kwargs
)

data = loader.load()

embeddings = CohereEmbeddings(model="embed-multilingual-v3.0", cohere_api_key ="iF6RFJiNfggDlFuIKvRh12Po5oTfjqXdvvOxflvU",user_agent="my-app")

db = Chroma.from_documents(
    documents=data,
    embedding=embeddings
    ,persist_directory="./chroma_db"
)

100%|██████████| 100/100 [00:00<00:00, 456.49it/s]


In [ ]:

from langchain_cohere import CohereEmbeddings
from langchain_cohere import ChatCohere
from langchain_cohere import CohereRerank
from langchain.vectorstores import Chroma
from langchain.retrievers import ContextualCompressionRetriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema.messages import HumanMessage, AIMessage
from langchain.chains import create_history_aware_retriever

llm = ChatCohere(cohere_api_key="iF6RFJiNfggDlFuIKvRh12Po5oTfjqXdvvOxflvU", model="command-r-plus-08-2024")
embeddings = CohereEmbeddings(model="embed-multilingual-v3.0", cohere_api_key ="iF6RFJiNfggDlFuIKvRh12Po5oTfjqXdvvOxflvU",user_agent="my-app") 
retriever = Chroma(persist_directory="./chroma_db", embedding_function=embeddings).as_retriever(search_kwargs={"k": 20})
reranker = CohereRerank(cohere_api_key="iF6RFJiNfggDlFuIKvRh12Po5oTfjqXdvvOxflvU", model="rerank-multilingual-v3.0")

system_prompt: str = (
        "Bạn là trợ lý trả lời văn bản luật tiếng Việt. "
        "Sử dụng ngữ cảnh được cung cấp để trả lời câu hỏi. "
        "\n\n"
        "Ngữ cảnh: \"{context}\" "
        "Bạn phải luôn trả lời từ nội dung, có thể cần phải dùng nhiều điều cho một lần trả lời, trích dẫn bắt buộc chính xác từng chữ. "
        "Nếu trả lời bên ngoài ngữ cảnh phải đề cập là bên ngoài ngữ cảnh"
        # "Nếu bạn không tìm thấy nội dung được hỏi từ ngữ cảnh, "
        # "nói \"Tôi không biết\". Không được nói điều nào về \"ngữ cảnh được cung cấp\" trong câu trả lời."
)
contextualize_q_system_prompt = (
        "Với lịch sử hội thoại và câu hỏi mới nhất của người dùng có thể đề cập đến "
        "ngữ cảnh trong lịch sử trò chuyện, hãy tổng hợp lại thành 1 câi hỏi độc lập "
        "mà không cần phải biết lịch sử hội thoại để hiểu được. Bạn không được trả lời câu hỏi. "
        "Chỉ cần tổng hợp lại thành 1 câu hỏi không cần lịch sử trò chuyện nếu cần, "
        "còn không thì trả về câu hỏi gốc."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
)
prompt: ChatPromptTemplate = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
#            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
)
qa_chain = create_stuff_documents_chain(llm, prompt)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=reranker,
    base_retriever=retriever
)

rag_chain = create_retrieval_chain(compression_retriever, qa_chain)

store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)
def invoke_rag_with_history(user_query: str, session_id: str) -> dict:
    """
    Invokes RAG chain and updates chat history
    """
    # Get response from chain
    response = conversational_rag_chain.invoke(
        {"input": user_query},
        {"configurable": {"session_id": session_id}}
    )
    
    # Update chat history
    history = get_session_history(session_id)
    history.add_user_message(HumanMessage(content=user_query))
    history.add_ai_message(AIMessage(content=response["answer"]))
    return response

In [28]:
# session_id = "1"
# user_query =  "Tôi vừa hỏi gì"
# response = conversational_rag_chain.invoke({"input": user_query},{'configurable': {'session_id': session_id}})
response = invoke_rag_with_history("Tôi tên gì", "1")
print(response["answer"])
# store[session_id].add_user_message(HumanMessage(content=user_query))
# store[session_id].add_ai_message(AIMessage(content=response["answer"]))
# compressed_docs = compression_retriever.get_relevant_documents(user_query)
# print(compressed_docs)

# # Create the cohere rag retriever using the chat model
# rag = CohereRagRetriever(llm=llm, connectors=[])
# docs = rag.get_relevant_documents(
#     user_query,
#     documents=compressed_docs,
# )
# # Print the documents
# print("Documents:")
# for doc in docs[:-1]:
#     print(doc.metadata)
#     print("\n\n" + doc.page_content)
#     print("\n\n" + "-" * 30 + "\n\n")
# Print the final generation
# answer = docs[-1].page_content
# print("Answer:")
# print(answer)
# # Print the final citations
# citations = docs[-1].metadata['citations']
# print("Citations:")
# print(citations)

Xin lỗi, tôi không tìm thấy tên của bạn trong ngữ cảnh được cung cấp. Bạn có thể cho tôi biết tên của bạn là gì không?
